In [20]:
import pandas as pd
import numpy as np

In [21]:
interaction = pd.read_csv ('./try/interaction_table.csv', index_col=False)
interaction = interaction.iloc[:, [2,3]]

In [22]:
multi = pd.read_csv ('./try/multidata_table.csv', index_col=None)
receptor = multi[multi['receptor'] == 1]
receptor = receptor.iloc[:, [0,1]]
ligand = multi[multi['receptor'] == 0]
ligand = ligand.iloc[:, [0,1]]

In [23]:
r=interaction[interaction['multidata_1_id'].isin(receptor['id_multidata'])].copy()
#fix column order for l-r
r[['multidata_1_id', 'multidata_2_id']] = r[['multidata_2_id', 'multidata_1_id']]
l=interaction[interaction['multidata_1_id'].isin(ligand['id_multidata'])].copy()

In [24]:
i=pd.concat([l, r], ignore_index=True).copy()

In [25]:
gene_table = pd.read_csv ('./try/gene_table.csv', index_col=False)
gene_table = gene_table.iloc[:, [3,4]].copy()
protein_name = pd.read_csv ('./try/protein_table.csv', index_col=False)
protein_name = protein_name.iloc[:, [1,5]].copy()
complex_comp = pd.read_csv ('./try/complex_composition_table.csv', index_col=False)
complex_comp = complex_comp.iloc[:, [1,2]].copy()

In [26]:
i["ligand"] = ''
i["receptor"] = ''
i["l_adhesion_mol"] = ''
i["r_adhesion_mol"] = ''
i["l_complex"] = ''
i["r_complex"] = ''

In [27]:
df = pd.DataFrame(columns=['L', 'R'])
df["ligand"] = ''
df["receptor"] = ''
df["l_adhesion_mol"] = ''
df["r_adhesion_mol"] = ''

In [28]:
#tag adhesion molecules for rec
for x in range(len(i)):
    p_id = i.loc[x].multidata_1_id
    if p_id in receptor.id_multidata.values:
        i.at[x, 'r_adhesion_mol'] = 1
    else:
        i.at[x, 'r_adhesion_mol'] = 0

In [29]:
#tag adhesion molecules for lig
for x in range(len(i)):
    p_id = i.loc[x].multidata_2_id
    if p_id in ligand.id_multidata.values:
        i.at[x, 'l_adhesion_mol'] = 1
    else:
        i.at[x, 'l_adhesion_mol'] = 0

In [30]:
# assign gene symbols for ligand /// assign prot_names not gene symbols.
df = pd.DataFrame()
for x in range(len(i)):
    p_id = i.loc[x].multidata_1_id
    if p_id < 1282:
        symbol = gene_table.loc[gene_table['protein_id'] == p_id].hgnc_symbol.unique()
        p_name = protein_name.loc[protein_name['protein_multidata_id'] == p_id].protein_name
        p_name = p_name.str.replace('_HUMAN', '')
        i.at[x, 'ligand'] = list(p_name)[0]
        i.at[x, 'l_complex'] = 0
        #get those ligands that have more than one symbol in a seperate df
        #this part can be taken out as no need anymore?
        if len(symbol) > 1:
            entry=i.loc[x].to_frame().transpose()
            df = pd.concat([df,entry])
            df.at[x, 'ligand'] = symbol
    else:
        complex_pid = complex_comp.loc[complex_comp['complex_multidata_id'] == p_id].protein_multidata_id.values
        i.at[x, 'l_complex'] = 1
        #complex ids matching 1 or more than 1 need to be mapped
        if len(complex_pid) == 1:
            p_name = protein_name.loc[protein_name['protein_multidata_id'] == complex_pid[0]].protein_name
            p_name = p_name.str.replace('_HUMAN', '')
            i.at[x, 'ligand'] = list(p_name)[0]
            
        else:
            entry = list()
            for j in complex_pid:
                p_name = protein_name.loc[protein_name['protein_multidata_id'] == j].protein_name
                p_name = p_name.str.replace('_HUMAN', '')
                p_name = p_name.values[0]
                entry.append(p_name)
            i.at[x, 'ligand'] = '_'.join(entry)

In [31]:
for x in range(len(i)):
    p_id = i.loc[x].multidata_2_id
    if p_id < 1282:
        p_name = protein_name.loc[protein_name['protein_multidata_id'] == p_id].protein_name
        p_name = p_name.str.replace('_HUMAN', '')
        i.at[x, 'receptor'] = list(p_name)[0]
        i.at[x, 'r_complex'] = 0
    
    # complex mol
    else:
        complex_pid = complex_comp.loc[complex_comp['complex_multidata_id'] == p_id].protein_multidata_id.values
        i.at[x, 'r_complex'] = 1
        #complex ids matching 1 or more than 1 need to be mapped
        if len(complex_pid) == 1:
            p_name = protein_name.loc[protein_name['protein_multidata_id'] == complex_pid[0]].protein_name
            p_name = p_name.str.replace('_HUMAN', '')
            i.at[x, 'receptor'] = list(p_name)[0]
            
        else:
            entry = list()
            for j in complex_pid:
                p_name = protein_name.loc[protein_name['protein_multidata_id'] == j].protein_name
                p_name = p_name.str.replace('_HUMAN', '')
                p_name = p_name.values[0]
                entry.append(p_name)
            i.at[x, 'receptor'] = '_'.join(entry)

In [32]:
i.drop(['multidata_1_id', 'multidata_2_id'], axis=1, inplace=True)

In [33]:
i["pairs"]=i[["ligand", "receptor"]].apply("_".join, axis=1)

In [34]:
i

,ligand,receptor,l_adhesion_mol,r_adhesion_mol,l_complex,r_complex,pairs
0,CER1,WNT11,1,0,0,0,CER1_WNT11
1,CER1,WNT5B,1,0,0,0,CER1_WNT5B
2,SFRP5,WNT5B,1,0,0,0,SFRP5_WNT5B
3,SFRP4,WNT5B,1,0,0,0,SFRP4_WNT5B
4,SFRP1,WNT5B,1,0,0,0,SFRP1_WNT5B
...,...,...,...,...,...,...,...
2283,IL36G,IL1AP,0,0,0,0,IL36G_IL1AP
2284,I36RA,ILRL2,0,0,0,0,I36RA_ILRL2
2285,I36RA,IL1AP,0,0,0,0,I36RA_IL1AP
2286,IL36A,ILRL2,0,0,0,0,IL36A_ILRL2


In [35]:
i.loc[i['pairs'] == "DHRS9_PRGR"]

,ligand,receptor,l_adhesion_mol,r_adhesion_mol,l_complex,r_complex,pairs
2130,DHRS9,PRGR,0,0,0,0,DHRS9_PRGR


In [36]:
i.to_csv('try_raw.csv', index=False)